## 10.12 임포트 시 모듈 패치
기존 모듈 함수에 데코레이터를 적용 or 패치 (실제로 임포트되고 사용되었을 때만)

-> 모듈을 불러오는 시점에 응답하고 특정 동작 수행.

In [ ]:
#postimport.py

import importlib
import sys
from collections import defaultdict

_post_import_hooks = defaultdict(list)

class PostImportFinder:
    def __init__(self):
        self._skip = set()
        
    def find_module(self, fullname, path=None):
        if fullname in self._skip:
            return None
        self._skip.add(fullname)
        return PostImportLoader(self)
class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder
        
    def load_module(self, fullname):
        importlib.import_module(fullname)
        module = sys.modules[fullname]
        for func in _post_import_hooks[fullname]:
            func(module)
        self._finder._skip.remove(fullname)
        return module
    
def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)
        return func
    return decorate

sys.meta_path.insert(0, PostImportFinder()) # sys.meta_path 에 모듈을 찾기 위한 파인더 객체 리스트가 담겨 있음

In [ ]:
from postimport import when_imported

@when_imported('threading') #임포트할 때 실행할 처리 함수를 등록
def warn_threads(mod):
    print('Threads? ARe you crazy?')
    
import threading 
# result : Threads? ARe you crazy?

@when_imported : 임포트할 때 실행할 처리 함수를 등록
2. 데코레이터는 이미 불러온 모듈이 없는지 sys.modules 확인
3. 있으면 핸들러 호출
4. 아니면 핸들러를 _post_import_hooks 딕셔너리 리스트에 추가

## 12.1 스레드 시작과 정지

In [5]:
import time
def countdown(n):
    while n > 0 :
        print('T-minus', n)
        n -= 1
        time.sleep(3)

- 스레드는 시스템 레블 스레드(ex.POSIX thread, Windows thread) 로 실행됨 (호스트 운영체제에서 관리)

In [6]:
from threading import Thread
t = Thread(target=countdown, args=(10,)) #target은 스레드로 돌릴 함수, args는 입력 인자.
t.start() #스레드의 실행을 시작, 내부적으로 이 메소드가 호출되면 자신의 run() 메소드를 호출

T-minus 10


In [7]:
if t.is_alive():
    print('Still running')
else:
    print('Completed')

Still running
T-minus 4
T-minus 9
T-minus 8
T-minus 3
T-minus 7
T-minus 6
T-minus 2
T-minus 5
T-minus 1
T-minus 4
T-minus 3
T-minus 2
T-minus 1


- 데몬 스레드 설정(daemon=True) -> 백그라운드에서 실행되는 쓰레드로 메인 쓰레드가 종료되면 즉시 종료됨.

q백그라운드에서 영원히 실행하는 스레드는 데몬으로 만들 수 있ㅇ


In [8]:
t1 = Thread(target=countdown, args=(10,), daemon=True)
t1.start()

T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


- 스레드를 종료하고 싶을 때

In [9]:
class CountdownTask:
    def __init__(self):
        self._running = True
        
    def terminate(self):
        self._running = False
        
    
    def run(self, n):
        while self._running and n>0 :
            print('T-minus', n)
            n -= 1
            time.sleep(1)
c = CountdownTask()
t = Thread(target=c.run, args=(5,))
t.start()

T-minus 5


In [10]:
c.terminate() #종료 지시

- 전역 인터프리터 락으로 인해 파이썬은 인터프리터에서 동시에 하나의 스레드만 실행할 수 있음.

파이썬 스레드는 여러 cpu에 병렬적으로 동작하는 복잡한 작업에는 사용하지 않는 것이 좋음. -> 입출력 처리 / 실행 멈추는 작업을 수행하는 코드에서 실행하자

- thread 클래스를 상속 받아서 정의할 때

-> z코드와 threading 라이브러리 사이에 불필요한 의존성 생겨서 별로

In [11]:
class CountdownThread(Thread):
    def __init__(self, n):
        super().__init__()
        self.n = n
    # ★★ Thread를 구동하기 위해서는 함수명을 run으로 해야함
    def run(self):
        while self.n>0:
            print('T-minus', self.n)
            self.n -= 1
            time.sleep(2)

In [12]:
c = CountdownThread(5)
c.start()

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


## 12.5 락킹으로 데드락 피하기

다중 스레드 프로그램에서 스레드가 하나 이상의 락을 취득해야 하고 데드락을 피해야 할 때

데드락 : 스레드가 동시에 여러 가지 락을 취득하려 할 때 발생

    ex) 한 스레드가 첫 번째 락 취득했지만 두번째 락 취득을 막는다면 이 스레드로 인해 다른 스레드 진행 문제 생김

-> 모뜬 띾예 유일한 숫자를 부여하고 오름차순으로 락을 취득하도록 강제.

In [2]:
import threading
from contextlib import contextmanager

_local = threading.local() #스레드가 다른 스레드와 공유하지 않고 자신의 스코프 내에서만 사용할 변수를 여기에 정의할 수 있음

@contextmanager
def acquire(*locks):
    # 객체 식별자로 락 정렬 ***핵심***
    locks = sorted(locks, key=lambda x:id(x))
    
    #이미 취득한 락의 순서가 깨지지 않도록 주의.
    acquired=getattr(_local, 'acquired', [])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock order violation')
        
    acquired.extend(locks)
    _local.acquired = acquired
    
    try:
        for lock in locks:
            lock.acquire() #미리 취득한 락 확인하고 기존 락이 새롭게 취득하려는 객체 id보다 작은 값을 갖도록 강제.
        yield
    
    #취득한 반대 순서로 락 해제
    finally:
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

In [3]:
_local

In [9]:
x_lock = threading.Lock() # 락 객체를 얻음
y_lock = threading.Lock()

def thread_1():
    n = 100
    while n > 0:
        with acquire(x_lock, y_lock):
            print('Thread-1')
            n -= 1
            
def thread_2():
    m = 100
    while m > 0:
        with acquire(y_lock ,x_lock):
            print('Thread-2')
            m -= 1
            
t1 = threading.Thread(target=thread_1)
t1.daemon = True
t1.start()

t2 = threading.Thread(target=thread_2)
t2.daemon = True
t2.start()

Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
T

각 함수에서 락 취득 순서가 서로 다르게 진행되지만 데드락 없이 잘 동작.

In [10]:
id(x_lock)

139974340123424

In [11]:
id(y_lock)

139974340124104

In [12]:
x_lock = threading.Lock() # 락 객체를 얻음
y_lock = threading.Lock()

def thread_1():
    n = 100
    while n > 0:
        with acquire(x_lock):
            with acquire(y_lock):
                print('Thread-1')
                n -= 1
            
def thread_2():
    m = 100
    while m > 0:
        with acquire(y_lock):
            with acquire(x_lock):
                print('Thread-2')
                m -= 1
            
t1 = threading.Thread(target=thread_1)
t1.daemon = True
t1.start()

t2 = threading.Thread(target=thread_2)
t2.daemon = True
t2.start()

Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2
Thread-2


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-f1c8168d1b50>", line 10, in thread_1
    n -= 1
  File "/usr/lib/python3.5/contextlib.py", line 77, in __exit__
    self.gen.throw(type, value, traceback)
  File "<ipython-input-2-0ff11d442f23>", line 22, in acquire
    yield
  File "<ipython-input-12-f1c8168d1b50>", line 8, in thread_1
    with acquire(y_lock):
  File "/usr/lib/python3.5/contextlib.py", line 59, in __enter__
    return next(self.gen)
  File "<ipython-input-2-0ff11d442f23>", line 14, in acquire
    raise RuntimeError('Lock order violation')
RuntimeError: Lock order violation



## 12.9 gil 다루기

전역 인터프리터 락(Global Interpreter Lock, GIL)

인터프리터는 동시에 파이썬 스레드를 하나만 실행할 수 있도록 허용하는 GIL에 의해 보호 받음.

GIL은 cpu를 많이 사용하는 프로그램에만 영향이 있음.

1. 전체적으로 파이썬만 사용 - multiprocessing 모듈로 프로세스 풀을 만들고 co-processor처럼 사용

In [17]:
#CPU 계산을 많이 수행하는 함수
def some_work(args):
    ...
    return result

# 위 함수를 호출하는 스레드
def some_thread():
    while True:
        r = some_work(args)
        

1. 스레드가 cpu를 많이 사용하는 작업하면 이 작업을 풀에게 넘김
2. 풀은 이 작업을 다른 프로세스에서 실행중인 별도의 파이썬 인터프리터에게 전가.
3. 스레드가 결과를 기다리는 동안 GIL을 해제.

In [18]:
# 프로세스 풀
pool = None

#CPU 계산을 많이 수행하는 함수
def some_work(args):
    ...
    return result

# 위 함수를 호출하는 스레드
def some_thread():
    while True:
        r = some_work(args)
        
# 풀 초기화
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()

2) c 확장 프로그래밍에 집중. 계산 많은 작업을 c로 이동시키고 c 코드가 작업하는 동안 GIL을 해제. 15.7, 15.10 에서 더 자세한 내용

In [19]:
#c 코드에 다음과 같이 특별 매크로 삽입

# include "Python.h"
...

PyObject *pyfunc(PyObject *self, PyObject *args){
    ...
    Py_BEGIN_ALLOW_THREADS
    //Threaded C code
    ...
    Py_END_ALLOW_THREADS
}

SyntaxError: invalid syntax (<ipython-input-19-1178fe1d71f4>, line 6)